In [79]:
import plotly.graph_objects as go
import numpy as np
import os

# モデルのパスや埋め込み範囲などのパラメータの設定

# 22311	mammal.n.01 929	bird.n.01 25214	reptile.n.01　5671	plant.n.01　 12623	fish.n.01 26628 corgi.n.01 39316 poodle.n.01
# 26266	lion.n.01 27	dog.n.01 22311	mammal.n.01 26878	cat.n.01 28865	homo_sapiens.n.01 34022	cheetah.n.01 5484	big_cat.n.01 5483	leopard.n.02 28505	wolf.n.01 16709	fox.n.01

model_path = '/home/lab/eight/KGE-HAKE/models/HAKE_wn18rr_0'
embedding_range = 0.01
head_id = 27 #22311	mammal.n.01 16430	food.n.01 39316	poodle.n.01
tail_id = 22311	#27	dog.n.01 40630	pizza.n.01
num_bins = 250  # ヒストグラムのビンの数
model = 'AdjustHAKE'
# エンティティ名の辞書読み込み
entities_dict_file = '/home/lab/eight/KGE-HAKE/data/wn18rr_text/entities.dict'
entities_dict = {}
with open(entities_dict_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        entities_dict[int(parts[0])] = parts[1]

In [83]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# HeadとTailの座標計算
head = entity_embedding[head_id]
tail = entity_embedding[tail_id]

phase_head, mod_head = np.split(head, 2)
phase_tail, mod_tail = np.split(tail, 2)

mean_head_mod = np.mean(mod_head)
mean_head_tail = np.mean(mod_tail)

print(entities_dict[head_id],mean_head_mod)
print(entities_dict[tail_id],mean_head_tail)
print(entities_dict[head_id],mod_head)
print(entities_dict[tail_id],mod_tail)

dog.n.01 0.0004911791
mammal.n.01 7.4815257e-06
dog.n.01 [ 1.33527564e-02  3.54151689e-02  1.67479310e-02  5.10660838e-03
  2.53982674e-02 -7.95840751e-03 -2.60046888e-02  2.56710108e-02
 -1.62924023e-03  1.91738605e-02  1.34074371e-02  1.05087068e-02
 -1.30491483e-03 -1.90059263e-02 -1.11909183e-02  5.85806090e-03
  2.83760345e-03 -1.25394175e-02  9.82176233e-03 -1.72432009e-02
  1.79448780e-02  1.07603734e-02 -1.74785405e-02 -1.85901821e-02
  1.35201411e-02 -2.24366598e-02 -1.86800398e-02  1.93241332e-02
  1.02265561e-02  5.80981653e-03 -2.46220399e-02  1.20439671e-03
 -9.50390380e-03  9.83053725e-03  3.51074822e-02  1.27776628e-02
 -7.09447404e-03 -1.14265168e-02  8.80181778e-06 -2.47904914e-03
 -4.33476828e-03  1.24381492e-02  1.56510621e-02  2.84937490e-02
  2.55154595e-02 -9.52668674e-03  1.85416802e-03  2.40483298e-03
 -1.59165692e-02 -3.29545699e-02 -2.55182814e-02  2.23373761e-03
 -1.81103628e-02  1.74060389e-02  1.34236636e-02 -2.30466411e-03
 -6.03079796e-03 -5.79976197e-03 

In [81]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# HeadとTailの座標計算
head = entity_embedding[head_id]
tail = entity_embedding[tail_id]

phase_head, mod_head = np.split(head, 2)
phase_tail, mod_tail = np.split(tail, 2)

mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

fig = go.Figure()

# Head Entityのプロット
fig.add_trace(go.Scatter(x=x_head, y=y_head, mode='markers', name=entities_dict[head_id], text=[f'Head ID: {head_id}'] * len(x_head), opacity=0.8))
# Tail Entityのプロット
fig.add_trace(go.Scatter(x=x_tail, y=y_tail, mode='markers', name=entities_dict[tail_id], text=[f'Tail ID: {tail_id}'] * len(x_tail), opacity=0.8))
# HAKEテキストの追加
fig.add_annotation(
    text=model,  # 表示するテキスト
    xref="paper",  # x座標の参照元 (paperは図の左端、dataはデータ座標)
    yref="paper",  # y座標の参照元 (paperは図の下端、dataはデータ座標)
    x=0.5,  # x座標 (0.5は図の中央)
    y=1.1,  # y座標 (1.05は図の上端から5%上)
    showarrow=False,  # 矢印を非表示にする
    font=dict(size=20),  # テキストのフォントサイズ
)
# レイアウトの設定
fig.update_layout(
    title="",
    yaxis_title=" ",
    showlegend=True,
    width=550,  # 幅
    height=500,  # 高さ
    xaxis=dict(scaleanchor="y", scaleratio=1),  # x軸のアスペクト比を1:1に設定
    yaxis=dict(scaleanchor="x", scaleratio=1),  # y軸のアスペクト比を1:1に設定
)

# プロットの表示
fig.show()


In [82]:

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

# ログ変換と極座標への変換
mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

# ヒストグラムの作成
hist_head = np.histogram(mod_head, bins=bin_edges)
hist_tail = np.histogram(mod_tail, bins=bin_edges)

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()

# Head Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_head[1][:-1], y=hist_head[0], name="head "+entities_dict[head_id], opacity=1.0, width=0.05))

# Tail Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_tail[1][:-1], y=hist_tail[0], name="tail "+entities_dict[tail_id], opacity=1.0, width=0.05))

# レイアウトの設定
fig.update_layout(
    title='HAKE',
    xaxis=dict(title='modulus 絶対値の大きさ'),
    yaxis=dict(title='要素 [hm/tm]i のカウント'),
    barmode='overlay',  # HeadとTailのヒストグラムをオーバーレイ
)

# プロットの表示
fig.show()


NameError: name 'bin_edges' is not defined

In [ ]:

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

# 指定された区間で均等に分割
bin_edges = np.linspace(0, np.max([np.max(phase_head), np.max(phase_tail)]), num_bins + 1)

# ヒストグラムの作成
hist_head = np.histogram(phase_head, bins=bin_edges)
hist_tail = np.histogram(phase_tail, bins=bin_edges)

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()

# Head Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_head[1][:-1], y=hist_head[0], name="head "+entities_dict[head_id], opacity=0.5, width=0.05))

# Tail Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_tail[1][:-1], y=hist_tail[0], name="tail "+entities_dict[tail_id], opacity=0.5, width=0.05))

# レイアウトの設定
fig.update_layout(
    title='HAKE',
    xaxis=dict(title='modulus 絶対値の大きさ'),
    yaxis=dict(title='要素 [hm/tm]i のカウント'),
    barmode='overlay',  # HeadとTailのヒストグラムをオーバーレイ
)

# プロットの表示
fig.show()


In [ ]:
# カウントの合計数を計算
head_count = np.sum(hist_head[0])
tail_count = np.sum(hist_tail[0])
# 平均距離の計算

phase_head, mod_head = np.split(entity_embedding[head_id], 2)
mean_distance_tail = np.mean(np.sqrt(x_tail**2 + y_tail**2))


mod_head = np.abs(mod_head)
mod_head = np.mean(mod_head)
# 結果の出力
print(f"平均距離（Head）: {mod_head}")
print(f"平均距離（Tail）: {mean_distance_tail}")
print(f"表示Entity（Head）: {head_count}")
print(f"表示Entity（Tail）: {tail_count}")

平均距離（Head）: 0.008023824542760849
平均距離（Tail）: 5.070516586303711
表示Entity（Head）: 249
表示Entity（Tail）: 260


In [ ]:

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

print("entity_embedding shape:", phase_head.shape)
print(f"mod_head: {mod_head}")

entity_embedding shape: (500,)
mod_head: [ 1.16768377e-02 -1.55864833e-02 -9.80700250e-04 -8.56324006e-03
  4.79101017e-03 -6.92110218e-04 -1.09423725e-02 -4.62262565e-03
  1.36547061e-02  1.59294270e-02 -1.47689804e-02 -7.07425177e-03
  4.67805238e-03 -5.19322418e-03 -3.77315725e-03 -6.00257516e-03
 -2.76235468e-03  1.10576330e-02 -1.52663291e-02 -6.86345063e-03
 -1.46232406e-02  7.71332718e-03  1.33762099e-02  3.02734226e-03
 -7.23222177e-03  2.69650272e-03  8.58079270e-03 -1.31482631e-02
  7.44890422e-03  3.78199667e-03 -2.80034053e-03  1.24547565e-02
  6.93540415e-03  1.13744242e-03 -6.47743372e-03  5.78830857e-03
  2.14913045e-04  9.18487727e-04  6.15118956e-03 -6.91235950e-03
  1.35727436e-03  1.04353912e-02  8.90562835e-04  5.81865106e-03
  1.30387768e-02 -1.87234674e-03 -1.11178244e-02 -2.66330689e-03
 -1.01337535e-02 -8.20836285e-05  1.56244123e-02  1.59784146e-02
  7.72756350e-04  2.51072738e-03 -7.12440023e-03  1.24158310e-02
  1.15660019e-02 -1.28687127e-02  2.49557733e-03 

In [ ]:
# エンティティの埋め込みをロード
relation_embedding = np.load(os.path.join(model_path, 'relation_embedding.npy'))


print(f"relation_embedding: {relation_embedding[4],}")

relation_embedding: (array([ 0.00208791, -0.00047733, -0.00326881, ..., -0.12873876,
       -0.09408756, -0.14988665], dtype=float32),)
